In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# input
notebook_dir = "/content/drive/MyDrive/Colab Notebooks/"
pkl_dir = notebook_dir + "preprocessed/"
dataset_prefix = "thunderbird_10M."
prefix = "withlinetoken.supervised.40-60."
template_train_pkl = pkl_dir + dataset_prefix + prefix + "template.trainset.pkl"
structured_train_pkl = pkl_dir+ dataset_prefix + prefix + "structured.trainset.pkl"
template_test_pkl = pkl_dir + dataset_prefix + prefix + "template.testset.pkl"
structured_test_pkl = pkl_dir + dataset_prefix + prefix + "structured.testset.pkl"

batch_size = 32

seq_lines_len = 5
seq_lines_step = 1

sample_ratio = 0.3
desample_quantile = 0.9

epochs = 5
time_prefix = "06-03-23_08-26."
exp_prefix = f"{seq_lines_len}_lines-downsampling-normal-{sample_ratio}-Q{int(desample_quantile*100):d}."
pred_dir = notebook_dir + "test_prediction/"
test_predict_parquet = pred_dir + time_prefix + f"{exp_prefix}{prefix}-{epochs}.parquet"

In [3]:
import pandas as pd

template_train = pd.read_pickle(template_train_pkl)
template_test  = pd.read_pickle(template_test_pkl)
test_pred      = pd.read_parquet(test_predict_parquet)

In [4]:
test_pred["Predict"] = test_pred[["normal_prob", "abnormal_prob"]].values.argmax(axis=1)

In [5]:
def build_confusion_filters(df, pos = 0, label_col = "Label", pred_col = "Predict"):
    TP = (df[label_col] == pos) & (df[pred_col] == pos)
    FP = (df[label_col] != pos) & (df[pred_col] == pos)
    TN = (df[label_col] != pos) & (df[pred_col] != pos)
    FN = (df[label_col] == pos) & (df[pred_col] != pos)

    return {"TP": TP, "FP": FP, "TN": TN, "FN": FN, "positive-class": pos}

In [6]:
filter = build_confusion_filters(test_pred, pos = 0)

In [7]:
prep_test_struc_log = pd.read_pickle(structured_test_pkl)

In [8]:
test_pred

,label,normal_prob,abnormal_prob,Label,EventId-1,EventId-2,EventId-3,EventId-4,EventId-5,Predict
0,0,1.000000,3.447218e-08,0,d58a5b9e,b4c694e2,d58a5b9e,d58a5b9e,d58a5b9e,0
1,0,1.000000,7.204187e-08,0,b4c694e2,d58a5b9e,d58a5b9e,d58a5b9e,d58a5b9e,0
2,0,1.000000,7.204187e-08,0,d58a5b9e,d58a5b9e,d58a5b9e,d58a5b9e,d58a5b9e,0
3,0,1.000000,7.204187e-08,0,d58a5b9e,d58a5b9e,d58a5b9e,d58a5b9e,d58a5b9e,0
4,0,1.000000,7.204187e-08,0,d58a5b9e,d58a5b9e,d58a5b9e,d58a5b9e,b38b2805,0
...,...,...,...,...,...,...,...,...,...,...
5985071,0,0.999995,4.535677e-06,0,b4c694e2,c20208b4,70d85acb,c20208b4,70d85acb,0
5985072,0,0.999996,3.624758e-06,0,c20208b4,70d85acb,c20208b4,70d85acb,b38b2805,0
5985073,0,0.999977,2.292224e-05,0,70d85acb,c20208b4,70d85acb,b38b2805,70d85acb,0
5985074,0,0.999996,3.624758e-06,0,c20208b4,70d85acb,b38b2805,70d85acb,b4c694e2,0


In [18]:
test_pred[filter["FN"]]

,label,normal_prob,abnormal_prob,Label,EventId-1,EventId-2,EventId-3,EventId-4,EventId-5,Predict
1281352,0,1.382917e-24,1.000000,0,b38b2805,70d85acb,91a643d5,6380bcd4,92ceb755,1
1281362,0,6.223622e-26,1.000000,0,91a643d5,91a643d5,91a643d5,6380bcd4,6380bcd4,1
1281363,0,6.223622e-26,1.000000,0,91a643d5,91a643d5,6380bcd4,6380bcd4,6380bcd4,1
1281364,0,6.223622e-26,1.000000,0,91a643d5,6380bcd4,6380bcd4,6380bcd4,b38b2805,1
1343306,0,6.223622e-26,1.000000,0,91a643d5,91a643d5,91a643d5,6380bcd4,6380bcd4,1
...,...,...,...,...,...,...,...,...,...,...
5796251,0,8.021464e-26,1.000000,0,91a643d5,531242a3,9b0f164f,952f1f69,6380bcd4,1
5796252,0,8.021464e-26,1.000000,0,531242a3,9b0f164f,952f1f69,6380bcd4,b4ae0123,1
5936335,0,8.021464e-26,1.000000,0,460b41a5,531242a3,9b0f164f,952f1f69,6380bcd4,1
5936336,0,8.021464e-26,1.000000,0,531242a3,9b0f164f,952f1f69,6380bcd4,756b67c2,1
